In [23]:
import cohere  #LLM used for the development of the application
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import os
import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI, Request, HTTPException, Form
from langchain.prompts.few_shot import FewShotPromptTemplate #for creating prompts with few-shot examples
from langchain.prompts.prompt import PromptTemplate #for formatting the few-shot examples
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import json
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain_cohere import ChatCohere
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
import psycopg2 #type: ignore
import json

load_dotenv()
cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
DB_CONFIG = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
}

In [2]:
def get_db_connection():
    conn = psycopg2.connect(
        user=DB_CONFIG["user"],
        password=DB_CONFIG["password"],
        host=DB_CONFIG["host"],
        port=DB_CONFIG["port"],
    )
    return conn

In [3]:
### Convierte las preguntas y respuestas en un dataframe de python
def db_get_questions():
    conn = get_db_connection()
    cursor = conn.cursor()
    # query = ''' SELECT * FROM respuestas_usuarios '''
    query = '''
    SELECT pr.id_pregunta, pr.usuario, pr.contenido as pregunta_contenido, pr.categoria,
    rs.contenido as respuesta_contenido, rs.id_respuesta, rs.id_pregunta, pr.activa FROM preguntas as pr
    LEFT JOIN respuestas as rs on pr.id_pregunta = rs.id_pregunta
    '''
    # cursor.fetchall()
    preguntas = pd.read_sql(query,conn)
    print(cursor)
    conn.close()
    return preguntas
preguntas = db_get_questions()
preguntas

<cursor object at 0x000001A07BB0BA00; closed: 0>


C:\Users\raulg\AppData\Local\Temp\ipykernel_7820\2756870851.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


,id_pregunta,usuario,pregunta_contenido,categoria,respuesta_contenido,id_respuesta,id_pregunta,activa
0,1,usuario,¿Cuál es tu color favorito?,divulgacion,Rojo,1.0,1.0,True
1,1,usuario,¿Cuál es tu color favorito?,divulgacion,Azul,4.0,1.0,True
2,2,usuario,¿Has tenido relaciones sexuales las ultimas 42h?,divulgacion,Si,2.0,2.0,True
3,2,usuario,¿Has tenido relaciones sexuales las ultimas 42h?,divulgacion,No,6.0,2.0,True
4,3,usuario,¿Las relaciones que mantuviste fueron con prot...,divulgacion,No,7.0,3.0,True
5,3,usuario,¿Las relaciones que mantuviste fueron con prot...,divulgacion,Si,3.0,3.0,True
6,5,usuario,¿Has tenido parejas sexuales recurrentes tras ...,divulgacion,Si,8.0,5.0,False
7,5,usuario,¿Has tenido parejas sexuales recurrentes tras ...,divulgacion,No,9.0,5.0,False
8,1,usuario,¿Cuál es tu color favorito?,divulgacion,Otro color,5.0,1.0,True
9,10,usuario,¿Ha habido alguna ocasión en la que hayas podi...,divulgacion,Si,10.0,10.0,True


In [4]:
def db_user_context(id_sesion):
    conn = get_db_connection()
    cursor = conn.cursor()
    # query = ''' SELECT * FROM respuestas_usuarios '''
    cursor.execute('''
        SELECT *
        FROM respuestas_usuarios
        WHERE id_sesion = %s
        ORDER BY id_respuesta_usuario DESC LIMIT 1
        ''', (id_sesion,))
    desc = cursor.description
    column_names = [col[0] for col in desc]
    data = [dict(zip(column_names, row))  
            for row in cursor.fetchall()]
    conn.close()
    return data[0]
info_user= db_user_context("prueba_raul_limpiador")
info_user

{'id_respuesta_usuario': 733,
 'direccion_ip': '80.102.39.52',
 'tipo_usuario': 'usuario',
 'municipio': 'Madrid',
 'ccaa': 'Madrid',
 'conocer_felgtbi': 'Por RRSS',
 'data_date': datetime.datetime(2024, 12, 10, 17, 17, 37, 920377),
 'vih_usuario': 'no',
 'vih_diagnostico': '',
 'vih_tratamiento': '',
 'us_edad': 29,
 'us_pais_origen': 'España',
 'us_genero': 'Hombre',
 'us_orientacion': 'Bisexual',
 'us_situacion_afectiva': 'Casado',
 'us_hablado': 'No',
 'pro_ambito': None,
 'pro_especialidad': None,
 'pro_vih_profesional': None,
 'id_sesion': 'prueba_raul_limpiador'}

In [5]:
categoria = "divulgacion"
tipo_usuario = "usuario"
df_preguntas = db_get_questions()
df_preguntas['respuesta_contenido'].tolist()

<cursor object at 0x000001A07BB0BBC0; closed: 0>


C:\Users\raulg\AppData\Local\Temp\ipykernel_7820\2756870851.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


['Rojo',
 'Azul',
 'Si',
 'No',
 'No',
 'Si',
 'Si',
 'No',
 'Otro color',
 'Si',
 'No',
 'Hace 1 semana',
 'Hace 2 semanas',
 'Si',
 'No',
 'Si',
 'No',
 'Si',
 'No',
 None]

In [37]:
df_preguntas = db_get_questions()
lista_preguntas= df_preguntas[(df_preguntas['categoria']==categoria) & (df_preguntas['usuario']==tipo_usuario) & (df_preguntas['activa']==True)]['pregunta_contenido'].unique().tolist()
lista_respuestas = df_preguntas['respuesta_contenido'].tolist()


C:\Users\raulg\AppData\Local\Temp\ipykernel_12220\2756870851.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


<cursor object at 0x000001612C757060; closed: 0>


In [ ]:
def llm_limpiador(id_sesion, categoria):

    ### Sacamos contexto del usuario y las preguntas activas, antes de tocar por el LLM.
    info_user= db_user_context(id_sesion)
    id_respuesta_usuario=info_user['id_respuesta_usuario']
    direccion_ip=info_user['direccion_ip']
    tipo_usuario=info_user['tipo_usuario']
    municipio=info_user['municipio']
    ccaa=info_user['ccaa']
    conocer_felgtbi=info_user['conocer_felgtbi']
    data_date=info_user['data_date']
    vih_usuario=info_user['vih_usuario']
    vih_diagnostico=info_user['vih_diagnostico']
    vih_tratamiento=info_user['vih_tratamiento']
    us_edad=info_user['us_edad']
    us_pais_origen=info_user['us_pais_origen']
    us_genero=info_user['us_genero']
    us_orientacion=info_user['us_orientacion']
    us_situacion_afectiva=info_user['us_situacion_afectiva']
    us_hablado=info_user['us_hablado']
    pro_ambito=info_user['pro_ambito']
    pro_especialidad=info_user['pro_especialidad']
    pro_vih_profesional=info_user['pro_vih_profesional']

    ### Preguntas
    df_preguntas = db_get_questions()
    preguntas_raw = df_preguntas[(df_preguntas['categoria']==categoria) & (df_preguntas['usuario']==tipo_usuario) & (df_preguntas['activa']==True)]['pregunta_contenido'].unique().tolist()
    dict_preguntas_raw = {}
    for i, pregunta in enumerate(preguntas_raw):
        dict_preguntas_raw[i] = pregunta
    dict_preguntas_raw

    ### LLM
    cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
    llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0,model='command-r')

    ### Response Schema para parser
    response_schemas = [
        ResponseSchema(name="diccionario_preguntas", description= '''Preguntas finales, en formato diccionario de Python. La clave es el índice
                       de la pregunta original, y el valor la pregunta filtrada. Ejemplo: 0:(pregunta1), 1:(pregunta2)'''),
        ResponseSchema(name="message", description="Explica de forma muy concisa por qué has excluido o incluido cada pregunta.")
    ]
    output_parser = JsonOutputParser() 

    ### Template y ejecución
    template = ChatPromptTemplate([
        ("system", ''' Recibes una lista de preguntas en {preguntas}. Tu misión es filtrar esas preguntas en función del contexto que
         conozcas del usuario. Pregunta sobre el embarazo sólo a tipo Mujer, o mujer cis género, y pregunta sobre la próstata sólo a tipo Hombre, u hombre cis género.
        Si el usuario no tiene vih, elimina del listado todas las preguntas relativas al vih, bajo ninguna
         circunstancia debes preguntar sobre el vih si el usuario no tiene vih. Sólo debes preguntar sobre el vih si el usuario nos confirma que tiene vih.
        NUNCA, en ningún caso modifiques el texto de las preguntas. Sólo quitarás según consideres dado el contexto del usuario, pero NUNCA variarás su contenido.
         Siempre escribirás "vih" (siempre en minúsculas, nunca en mayúsculas), ya que estamos desetigmatizando sobre la enfermedad.
         Devuelve exactamente un JSON con dos claves: 
        - "diccionario_preguntas": Las preguntas finales como un diccionario de Python. La clave de la pregunta debe coincidir con el índice del listado de preguntas en {preguntas},
         mientras que el valor será el texto de la pregunta.
        - "message": Respuesta al usuario por parte del bot, explicando de forma detallada por qué has excluido o incluido cada pregunta.
        NUNCA, NUNCA, incluyas texto adicional fuera del JSON.
         '''),
        ("human", '''Soy tipo de usuario {tipo_usuario}.  Vivo en {municipio} en la comunidad {ccaa}. Tengo {us_edad} años y nací en {us_pais_origen}.
         Mi género es {us_genero} y mi orientación sexual es {us_orientacion}. Sobre mi situación afectiva, {us_situacion_afectiva}.
         {us_hablado} he hablado sobre si tengo vih con mi entorno. A la pregunta de si tengo vih, la respuesta es {vih_usuario}, y, en caso de tenerlo, me dieron
         el diagnóstico hace {vih_diagnostico}, y si sigo tratamiento es desde hace {vih_tratamiento}. Si soy tipo de usuario "profesional",
        mi ámbito es {pro_ambito}, mi especialidad {pro_especialidad} y {pro_vih_profesional} he tratado a un afectado por vih.
          ''')],
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
        )
    prompt_value = template.invoke({"preguntas":preguntas_raw,"tipo_usuario":tipo_usuario,"municipio":municipio, "ccaa":ccaa, "conocer_felgtbi":conocer_felgtbi, "vih_usuario":vih_usuario,
                                    "vih_diagnostico":vih_diagnostico, "vih_tratamiento":vih_tratamiento, "us_edad": us_edad, "us_pais_origen":us_genero,
                                    "us_genero":us_genero,"us_orientacion":us_orientacion, "us_situacion_afectiva": us_situacion_afectiva,
                                    "us_hablado":us_hablado,"pro_ambito":pro_ambito,"pro_especialidad":pro_especialidad,
                                    "pro_vih_profesional":pro_vih_profesional})
    
    print(dict_preguntas_raw)
    print(us_genero)
    print(vih_usuario)
    response = llm.invoke(prompt_value)
    print(response.content)
    parsed_output = output_parser.parse(response.content)
    

    return parsed_output


In [34]:
preguntas = llm_limpiador('prueba_raul_limpiador', 'divulgacion')
dict_preg_resp = {}
for pregunta in preguntas['diccionario_preguntas'].keys():
    resp_list = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()
    dict_preg_resp[pregunta] = resp_list
dict_preg_resp

C:\Users\raulg\AppData\Local\Temp\ipykernel_7820\2756870851.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


<cursor object at 0x000001A07D978660; closed: 0>


TypeError: JsonOutputParser.get_format_instructions() takes 1 positional argument but 2 were given

In [30]:
preguntas

'```json\n{\n  "diccionario_preguntas": {\n    "0": "¿Cu\\u00e1l es tu color favorito?",\n    "1": "¿Has tenido relaciones sexuales las ultimas 42h?",\n    "2": "¿Las relaciones que mantuviste fueron con protecci\\u00f3n?",\n    "3": "¿Cu\\u00f3ndo ha sido tu \\u00fltimo periodo?",\n    "4": "¿Has tenido revisi\\u00f3n de la pr\\u00f3stata?",\n    "7": "¿Est\\u00e1s embarazada?"\n  },\n  "message": "Hemos eliminado algunas preguntas porque, seg\\u00fan el contexto que proporcionaste, no son pertinentes para ti. No tienes el VIH, por lo que elimin\\u00e9 todas las preguntas relacionadas con \\u00e9l. Tambi\\u00e9n has indicado que eres un hombre, por lo que la pregunta sobre el periodo menstrual y el embarazo no son apropiadas en tu caso."\n}\n```'

In [29]:
dict_preg_resp = {}
for pregunta in preguntas['diccionario_preguntas'].keys():
    resp_list = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()
    dict_preg_resp[pregunta] = resp_list
dict_preg_resp

{'¿Cuál es tu color favorito?': ['Rojo', 'Azul', 'Otro color'],
 '¿Has tenido relaciones sexuales las ultimas 42h?': ['Si', 'No'],
 '¿Las relaciones que mantuviste fueron con proteccion?': ['No', 'Si'],
 '¿Ha habido alguna ocasión en la que hayas podido contagiar a alguien el vih?': [None],
 '¿Has tenido revisión de la próstata?': [None]}

In [20]:
preguntas_raw = df_preguntas[(df_preguntas['categoria']=="divulgacion") & (df_preguntas['usuario']=="usuario") & (df_preguntas['activa']==True)]['pregunta_contenido'].unique()
preguntas_raw
dict_preguntas_raw = {}
for i, pregunta in enumerate(preguntas_raw):
    dict_preguntas_raw[i] = pregunta
dict_preguntas_raw

{0: '¿Cuál es tu color favorito?',
 1: '¿Has tenido relaciones sexuales las ultimas 42h?',
 2: '¿Las relaciones que mantuviste fueron con proteccion?',
 3: '¿Ha habido alguna ocasión en la que hayas podido contagiar a alguien el vih?',
 4: '¿Cuándo ha sido tu último periodo?',
 5: '¿Estás embarazada?',
 6: '¿Has notado síntomas desde que te diagnosticaron vih?',
 7: '¿Has tenido revisión de la próstata?'}

### Ejemplo con Álex

In [77]:
### Ejemplo de Álex
def llm_limpiador(id_sesion, categoria):
    # Obtener contexto del usuario
    info_user = db_user_context(id_sesion)
    tipo_usuario = info_user['tipo_usuario']
    municipio = info_user['municipio']
    ccaa = info_user['ccaa']
    vih_usuario = info_user['vih_usuario']
    us_edad = info_user['us_edad']
    us_pais_origen = info_user['us_pais_origen']
    us_genero = info_user['us_genero']
    us_orientacion = info_user['us_orientacion']
    us_situacion_afectiva = info_user['us_situacion_afectiva']
    us_hablado = info_user['us_hablado']
    pro_ambito = info_user['pro_ambito']
    pro_especialidad = info_user['pro_especialidad']
    pro_vih_profesional = info_user['pro_vih_profesional']

    # Obtener preguntas activas
    df_preguntas = db_get_questions()
    preguntas_unicas = df_preguntas[
        (df_preguntas['categoria'] == categoria) &
        (df_preguntas['usuario'] == tipo_usuario) &
        (df_preguntas['activa'] == True)
    ][['pregunta_contenido','id_pregunta']].drop_duplicates()
    lista_preguntas = preguntas_unicas['pregunta_contenido'].tolist()
    lista_ids = preguntas_unicas['id_pregunta'].iloc[:,0].tolist()
    dict_preguntas = {}
    for i in range(len(lista_preguntas)):
        dict_preguntas[lista_ids[i]] = lista_preguntas[i]
    dict_preguntas

    # Configuración del LLM
    cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
    llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0, model='command-r')

    # Template del prompt
    prompt = f"""
    Recibes un diccionario de preguntas: {dict_preguntas}. La clave es el id de la pregunta, y el valor es el texto de la pregunta.
    Tu misión es filtrar esas preguntas en función del contexto del usuario:
    - Si el usuario no tiene vih, elimina preguntas relacionadas con el vih.
    - Pregunta sobre embarazo solo si el género es "mujer" o "mujer cisgénero".
    - Pregunta sobre próstata solo si el género es "hombre" o "hombre cisgénero".

    NO modifiques el texto de las preguntas, solo elimina según el contexto. NUNCA modifiques el texto de las preguntas.

    Información del usuario:
    Tipo: {tipo_usuario}, Municipio: {municipio}, CCAA: {ccaa}, Edad: {us_edad}, País de origen: {us_pais_origen},
    Género: {us_genero}, Orientación: {us_orientacion}, Situación afectiva: {us_situacion_afectiva}, Ha hablado sobre vih: {us_hablado},
    Tiene vih: {vih_usuario}, Profesional (ámbito: {pro_ambito}, especialidad: {pro_especialidad}, experiencia con vih: {pro_vih_profesional}).

    Devuelve un diccionario en Python con las preguntas filtradas, en este formato:
    {{
        "preguntas_filtradas": {{ id de la pregunta original: "texto de la pregunta" }},
        "message": "Explicación de por qué se incluyeron/excluyeron las preguntas."
    }}
    """

    # Invocar el modelo
    response = llm.invoke(prompt)
    print("Respuesta del modelo:", response.content)

    # Procesar la respuesta como un diccionario
    try:
        result = eval(response.content.strip())
    except (SyntaxError, ValueError) as e:
        print(f"Error procesando la respuesta: {e}")
        result = {
            "preguntas_filtradas": {},
            "message": "Error al procesar la respuesta del modelo. Verifica el prompt o el formato de salida."
        }

    return result


In [78]:
preguntas = llm_limpiador('prueba_raul_limpiador', 'divulgacion')

preguntas

C:\Users\raulg\AppData\Local\Temp\ipykernel_7820\2756870851.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


<cursor object at 0x000001A07D5552A0; closed: 0>
Respuesta del modelo: {
    "preguntas_filtradas": {
        1: "¿Cuál es tu color favorito?",
        7: "¿Has tenido revisión de la próstata?",
        8: "¿Cuándo ha sido tu último periodo?"
    },
    "message": "Se han filtrado las preguntas relacionadas con el VIH porque el usuario no lo tiene. La pregunta sobre la próstata se incluye por el género del usuario, y la del periodo porque es un hombre cisgénero."
}


{'preguntas_filtradas': {1: '¿Cuál es tu color favorito?',
  7: '¿Has tenido revisión de la próstata?',
  8: '¿Cuándo ha sido tu último periodo?'},
 'message': 'Se han filtrado las preguntas relacionadas con el VIH porque el usuario no lo tiene. La pregunta sobre la próstata se incluye por el género del usuario, y la del periodo porque es un hombre cisgénero.'}

In [63]:
preguntas_unicas = df_preguntas[
        (df_preguntas['categoria'] == categoria) &
        (df_preguntas['usuario'] == tipo_usuario) &
        (df_preguntas['activa'] == True)
    ][['pregunta_contenido','id_pregunta']].drop_duplicates()
lista_preguntas = preguntas_unicas['pregunta_contenido']
type(lista_preguntas)

pandas.core.series.Series

In [71]:
preguntas_unicas = df_preguntas[
        (df_preguntas['categoria'] == categoria) &
        (df_preguntas['usuario'] == tipo_usuario) &
        (df_preguntas['activa'] == True)
    ][['pregunta_contenido','id_pregunta']].drop_duplicates()
lista_preguntas = preguntas_unicas['pregunta_contenido'].tolist()
lista_ids = preguntas_unicas['id_pregunta'].iloc[:,0].tolist()
dict_preguntas = {}
for i in range(len(lista_preguntas)):
    dict_preguntas[lista_ids[i]] = lista_preguntas[i]
dict_preguntas


{1: '¿Cuál es tu color favorito?',
 2: '¿Has tenido relaciones sexuales las ultimas 42h?',
 3: '¿Las relaciones que mantuviste fueron con proteccion?',
 10: '¿Ha habido alguna ocasión en la que hayas podido contagiar a alguien el vih?',
 8: '¿Cuándo ha sido tu último periodo?',
 7: '¿Has tenido revisión de la próstata?',
 9: '¿Has notado síntomas desde que te diagnosticaron vih?',
 6: '¿Estás embarazada?'}

In [38]:
df_preguntas

,id_pregunta,usuario,pregunta_contenido,categoria,respuesta_contenido,id_respuesta,id_pregunta,activa
0,1,usuario,¿Cuál es tu color favorito?,divulgacion,Rojo,1.0,1.0,True
1,1,usuario,¿Cuál es tu color favorito?,divulgacion,Azul,4.0,1.0,True
2,2,usuario,¿Has tenido relaciones sexuales las ultimas 42h?,divulgacion,Si,2.0,2.0,True
3,2,usuario,¿Has tenido relaciones sexuales las ultimas 42h?,divulgacion,No,6.0,2.0,True
4,3,usuario,¿Las relaciones que mantuviste fueron con prot...,divulgacion,No,7.0,3.0,True
5,3,usuario,¿Las relaciones que mantuviste fueron con prot...,divulgacion,Si,3.0,3.0,True
6,5,usuario,¿Has tenido parejas sexuales recurrentes tras ...,divulgacion,Si,8.0,5.0,False
7,5,usuario,¿Has tenido parejas sexuales recurrentes tras ...,divulgacion,No,9.0,5.0,False
8,1,usuario,¿Cuál es tu color favorito?,divulgacion,Otro color,5.0,1.0,True
9,10,usuario,¿Ha habido alguna ocasión en la que hayas podi...,divulgacion,Si,10.0,10.0,True


In [44]:
pregunta1 = "¿Has notado algun sintoma desde el diagnostico?"
df_preguntas[df_preguntas["pregunta_contenido"] == pregunta1]["respuesta_contenido"].tolist()

[]

In [9]:
df_preguntas = db_get_questions()
df_preguntas

<cursor object at 0x000001612B1AFA00; closed: 0>


C:\Users\raulg\AppData\Local\Temp\ipykernel_12220\2756870851.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  preguntas = pd.read_sql(query,conn)


,id_pregunta,usuario,pregunta_contenido,categoria,respuesta_contenido,id_respuesta,id_pregunta,activa
0,1,usuario,¿Cuál es tu color favorito?,divulgacion,Rojo,1.0,1.0,True
1,1,usuario,¿Cuál es tu color favorito?,divulgacion,Azul,4.0,1.0,True
2,2,usuario,¿Has tenido relaciones sexuales las ultimas 42h?,divulgacion,Si,2.0,2.0,True
3,2,usuario,¿Has tenido relaciones sexuales las ultimas 42h?,divulgacion,No,6.0,2.0,True
4,3,usuario,¿Las relaciones que mantuviste fueron con prot...,divulgacion,No,7.0,3.0,True
5,3,usuario,¿Las relaciones que mantuviste fueron con prot...,divulgacion,Si,3.0,3.0,True
6,5,usuario,¿Has tenido parejas sexuales recurrentes tras ...,divulgacion,Si,8.0,5.0,False
7,5,usuario,¿Has tenido parejas sexuales recurrentes tras ...,divulgacion,No,9.0,5.0,False
8,1,usuario,¿Cuál es tu color favorito?,divulgacion,Otro color,5.0,1.0,True
9,10,usuario,¿Ha habido alguna ocasión en la que hayas podi...,divulgacion,None,NaN,NaN,True


{'¿Cuál es tu color favorito?': ['Rojo', 'Azul', 'Otro color'],
 '¿Has tenido relaciones sexuales las últimas 42h?': [],
 '¿Las relaciones que mantuviste fueron con protección?': [],
 '¿Ha habido alguna ocasión en la que hayas podido contagiar a alguien el vih?': [None],
 '¿Has tenido revisión de la próstata?': [None]}

In [ ]:
dict_preg_resp = {}
for pregunta in df_preguntas["pregunta_contenido"].drop_duplicates():
     resp_list = df_preguntas[df_preguntas["pregunta_contenido"] == pregunta]["respuesta_contenido"].tolist()
     dict_preg_resp[pregunta] = resp_list
dict_preg_resp

In [ ]:
preguntas_raw

['¿Cuál es tu color favorito?',
 '¿Has tenido relaciones sexuales las ultimas 42h?',
 '¿Las relaciones que mantuviste fueron con proteccion?',
 '¿Ha habido alguna ocasión en la que hayas podido contagiar a alguien el vih?',
 '¿Cuándo ha sido tu último periodo?',
 '¿Estás embarazada?',
 '¿Has notado síntomas desde que te diagnosticaron vih?',
 '¿Has tenido revisión de la próstata?']

In [130]:
listado_preguntas = respuesta['listado_preguntas']

In [132]:
lista_preguntas[1]

'¿Has tenido relaciones sexuales las ultimas 42h?'

In [ ]:
listta

In [91]:
def llm_prueba(id_sesion, categoria):
    ### Sacamos contexto del usuario y las preguntas activas, antes de tocar por el LLM.
    info_user= db_user_context(id_sesion)
    id_respuesta_usuario=info_user['id_respuesta_usuario']
    direccion_ip=info_user['direccion_ip']
    tipo_usuario=info_user['tipo_usuario']
    municipio=info_user['municipio']
    ccaa=info_user['ccaa']
    conocer_felgtbi=info_user['conocer_felgtbi']
    data_date=info_user['data_date']
    vih_usuario=info_user['vih_usuario']
    vih_diagnostico=info_user['vih_diagnostico']
    vih_tratamiento=info_user['vih_tratamiento']
    us_edad=info_user['us_edad']
    us_pais_origen=info_user['us_pais_origen']
    us_genero=info_user['us_genero']
    us_orientacion=info_user['us_orientacion']
    us_situacion_afectiva=info_user['us_situacion_afectiva']
    us_hablado=info_user['us_hablado']
    pro_ambito=info_user['pro_ambito']
    pro_especialidad=info_user['pro_especialidad']
    pro_vih_profesional=info_user['pro_vih_profesional']
    locals().update(info_user)
    print(id_respuesta_usuario)
    print(conocer_felgtbi)
    print(us_orientacion)
llm_prueba('prueba_raul_limpiador', 'divulgacion')


733
Por RRSS
Bisexual
